In [11]:
import json
from datetime import datetime

def convert_to_rag_format(crawled_jobs, output_file="rag_data.jsonl"):
    """
    Konvertiert gecrawlte Jobs in das JSONL-Format für ein RAG-System.

    Args:
        crawled_jobs (list): Liste der gecrawlten Stellenangebote.
        output_file (str): Dateiname für die JSONL-Ausgabe.
    """
    rag_data = []

    for job in crawled_jobs:
        # Titel und URL aus dem Job übernehmen
        title = job.get("title", "Unbekannter Titel")
        url = job.get("url", "Keine URL")
        description = job.get("description", "Keine Beschreibung")
        
        # Eintrag für RAG-System erstellen
        entry = {
            "prompt": f"Was enthält die Seite '{title}'?",
            "completion": description,
            "meta": {
                "title": title,
                "url": url,
                "timestamp": datetime.utcnow().isoformat()
            }
        }
        rag_data.append(entry)

    # Daten in JSONL-Datei speichern
    with open(output_file, "w", encoding="utf-8") as file:
        for entry in rag_data:
            file.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Daten erfolgreich in {output_file} gespeichert.")


In [12]:
input_file = "jobs_data.json"
output_file = "fixed_jobs_data.json"

try:
    with open(input_file, "r", encoding="utf-8") as file:
        raw_data = file.read()

    # Ersetze einfache Anführungszeichen durch doppelte
    fixed_data = raw_data.replace("'", '"')

    # Reparierte Datei speichern
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(fixed_data)

    print(f"Reparierte Datei wurde in {output_file} gespeichert.")
except Exception as e:
    print(f"Fehler bei der Reparatur: {e}")


Reparierte Datei wurde in fixed_jobs_data.json gespeichert.


In [13]:
input_file = "jobs_data.json"
output_file = "repaired_jobs_data.json"

try:
    repaired_data = []
    current_object = {}
    
    with open(input_file, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line.startswith("{"):
                # Start eines neuen Objekts
                current_object = {}
            elif line.startswith("}"):
                # Ende eines Objekts -> Objekt speichern
                repaired_data.append(current_object)
            else:
                # Versuche Schlüssel-Wert-Paare zu erkennen
                if ":" in line:
                    key, value = line.split(":", 1)
                    key = key.strip().strip('"')  # Entfernt unnötige Leerzeichen/Anführungszeichen
                    value = value.strip().strip(',').strip('"')  # Entfernt Komma/Anführungszeichen
                    current_object[key] = value
    
    # Reparierte Daten als gültige JSON-Liste speichern
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(repaired_data, file, ensure_ascii=False, indent=4)

    print(f"Reparierte Datei erfolgreich gespeichert unter: {output_file}")
except Exception as e:
    print(f"Fehler bei der Reparatur: {e}")


Reparierte Datei erfolgreich gespeichert unter: repaired_jobs_data.json


In [14]:
import json

repaired_file = "repaired_jobs_data.json"

with open(repaired_file, "r", encoding="utf-8") as file:
    crawled_jobs = json.load(file)

print(f"{len(crawled_jobs)} Jobs erfolgreich geladen.")


250 Jobs erfolgreich geladen.


In [15]:
output_file = "rag_data.jsonl"
convert_to_rag_format(crawled_jobs, output_file=output_file)
print(f"Daten erfolgreich in {output_file} konvertiert.")


Daten erfolgreich in rag_data.jsonl gespeichert.
Daten erfolgreich in rag_data.jsonl konvertiert.


In [16]:
import pandas as pd

df = pd.DataFrame(crawled_jobs)
print(df.head())


                                               title  \
0                          Technical Trainer (m/w/d)   
1                        Cloud Engineer (AWS) Senior   
2                      Senior Cloud Engineer (Azure)   
3                           Cloud Engineer (AWS) Mid   
4  Werkstudent (m/w/d) Sustainability / Corporate...   

                         company  \
0   Körber Pharma Packaging GmbH   
1  Körber Porto, Unipessoal Lda.   
2  Körber Porto, Unipessoal Lda.   
3  Körber Porto, Unipessoal Lda.   
4                      Körber AG   

                                            location  \
0  Schloß Holte-Stukenbrock, Nordrhein-Westfalen, DE   
1                                          Porto, PT   
2                                          Porto, PT   
3                                          Porto, PT   
4                                    Hamburg, HH, DE   

                                                 url  \
0  https://jobs.koerber.com/pharma/job/Schlo%C3%9... 

In [17]:
import json

def validate_jsonl(file_path):
    """
    Validiert eine JSONL-Datei für die Verarbeitung zu Chunks und Vektorspeicher.

    Args:
        file_path (str): Pfad zur JSONL-Datei.

    Returns:
        bool: True, wenn die Datei gültig ist und verarbeitet werden kann.
        list: Liste der validierten Datensätze.
    """
    valid = True
    records = []

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for line_num, line in enumerate(f, start=1):
                try:
                    record = json.loads(line.strip())
                    # Prüfen, ob notwendige Schlüssel vorhanden sind
                    if not all(key in record for key in ["prompt", "completion", "meta"]):
                        print(f"Fehlender Schlüssel in Zeile {line_num}.")
                        valid = False
                        continue
                    
                    # Prüfen, ob meta die Schlüssel 'title' und 'url' enthält
                    if not all(key in record["meta"] for key in ["title", "url"]):
                        print(f"Fehlender Metadaten-Schlüssel in Zeile {line_num}.")
                        valid = False
                        continue
                    
                    # Prüfen, ob completion nicht leer ist
                    if not record["completion"].strip():
                        print(f"Leerer Textinhalt in Zeile {line_num}.")
                        valid = False
                        continue

                    records.append(record)
                except json.JSONDecodeError:
                    print(f"JSON-Fehler in Zeile {line_num}.")
                    valid = False
    except FileNotFoundError:
        print(f"Datei {file_path} nicht gefunden.")
        return False, []

    return valid, records


# Beispielprüfung
file_path = "rag_data.jsonl"
is_valid, data = validate_jsonl(file_path)

if is_valid:
    print(f"Die Datei {file_path} ist gültig und enthält {len(data)} Datensätze.")
else:
    print(f"Die Datei {file_path} ist ungültig.")


JSON-Fehler in Zeile 1.
Die Datei rag_data.jsonl ist ungültig.


In [18]:
def check_chunk_suitability(data, chunk_size=500):
    """
    Prüft, ob die Datensätze groß genug sind, um in Chunks aufgeteilt zu werden.

    Args:
        data (list): Liste der Datensätze aus der JSONL-Datei.
        chunk_size (int): Minimale Größe eines Chunks.

    Returns:
        bool: True, wenn die Datensätze geeignet sind.
    """
    suitable = True
    for idx, record in enumerate(data):
        if len(record["completion"]) < chunk_size:
            print(f"Datensatz {idx + 1} ist zu kurz für Chunks (nur {len(record['completion'])} Zeichen).")
            suitable = False
    return suitable


if is_valid:
    if check_chunk_suitability(data):
        print("Alle Datensätze sind groß genug für Chunking.")
    else:
        print("Einige Datensätze sind zu klein für Chunking.")


In [19]:
def validate_and_fix_jsonl(file_path):
    """
    Validiert eine JSONL-Datei Zeile für Zeile und korrigiert fehlerhafte JSON-Einträge, falls möglich.

    Args:
        file_path (str): Pfad zur JSONL-Datei.

    Returns:
        list: Liste der validen Datensätze.
    """
    valid_records = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, start=1):
            try:
                record = json.loads(line.strip())
                valid_records.append(record)
            except json.JSONDecodeError as e:
                print(f"JSON-Fehler in Zeile {line_num}: {e}")
                # Optional: Protokolliere die fehlerhaften Zeilen oder versuche eine Korrektur
    return valid_records

# Beispiel: Validierung
file_path = "rag_data.jsonl"
valid_data = validate_and_fix_jsonl(file_path)

if valid_data:
    print(f"Validierte {len(valid_data)} Datensätze erfolgreich.")
else:
    print("Keine gültigen Datensätze gefunden.")


JSON-Fehler in Zeile 1: Invalid \escape: line 1 column 261 (char 260)
Validierte 249 Datensätze erfolgreich.


In [20]:
def reformat_jsonl(input_path, output_path):
    """
    Reformatiert eine JSONL-Datei, indem fehlerhafte Zeilen übersprungen werden.

    Args:
        input_path (str): Pfad zur Eingabedatei.
        output_path (str): Pfad zur Ausgabedatei.

    Returns:
        int: Anzahl der validierten Zeilen.
    """
    valid_count = 0
    with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
        for line_num, line in enumerate(infile, start=1):
            try:
                record = json.loads(line.strip())
                json.dump(record, outfile, ensure_ascii=False)
                outfile.write("\n")
                valid_count += 1
            except json.JSONDecodeError:
                print(f"Fehlerhafte Zeile {line_num} übersprungen.")
    return valid_count

# Neuformatieren der JSONL-Datei
input_path = "rag_data.jsonl"
output_path = "cleaned_rag_data.jsonl"
valid_lines = reformat_jsonl(input_path, output_path)

print(f"Reformatierung abgeschlossen. {valid_lines} Zeilen erfolgreich gespeichert.")


Fehlerhafte Zeile 1 übersprungen.
Reformatierung abgeschlossen. 249 Zeilen erfolgreich gespeichert.
